In [1]:
import os
import sys
import pandas as pd
import numpy as np
import tensorflow as tf
# Append source directory to path
sys.path.append("../../src/")
import hipopy.hipopy
import ROOT
from particle import Particle
import matplotlib.pyplot as plt
# Import custom modules
from ECALDataReader import ECALDataReader
from TrainData import load_unzip_data
from loss_functions import CustomLoss, AttractiveLossMetric, RepulsiveLossMetric, CowardLossMetric, NoiseLossMetric
from ModelEcalPlotter import ModelEcalPlotter
from Evaluator import Evaluator
from global_params import *
plt.style.use('science')
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})
import math
from matplotlib.gridspec import GridSpec

2024-10-08 14:53:05.297185: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-08 14:53:06.155899: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-08 14:53:06.841885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-08 14:53:07.393612: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-08 14:53:07.538906: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-08 14:53:08.807273: I tensorflow/core/platform/cpu_feature_gu

This analysis notebook lets us study the *before* and *after* of the object condensation...

In [37]:
# Obtain relevant files
def get_files(project_name, suffix):
    dst_dir = f'../../projects/{project_name}/dst/'
    predict_dir = f'../../projects/{project_name}/predict/'
    training_dir = f'../../projects/{project_name}/training/'
    file_basename_prefix = "clasdispr.00.e10.600.emn0.75tmn.09.xs68.34nb.dis"
    file_basename = f'{file_basename_prefix}.{suffix}'

    coatjava_hipofile = f'{dst_dir}/{file_basename}.hipo'
    objectcondensation_hipofile = f'{predict_dir}/{file_basename}_ML.hipo'
    objectcondensation_h5file = f'{training_dir}/{file_basename}.h5'
    return coatjava_hipofile, objectcondensation_hipofile, objectcondensation_h5file

# Extract model training data
def get_model_inputs(h5file):
    X, y, misc = load_unzip_data(h5file)
    return X, y, misc

# Get model evaluator
def get_evaluator(h5file="", model_load_path="", tB=0.5, tD=0.5, offset=0):
    X, y, misc = get_model_inputs(h5file)
    
    # Load the model
    model = tf.keras.models.load_model(model_load_path, custom_objects={
        'CustomLoss': CustomLoss, 
        'AttractiveLossMetric': AttractiveLossMetric,
        'RepulsiveLossMetric': RepulsiveLossMetric,
        'CowardLossMetric': CowardLossMetric,
        'NoiseLossMetric': NoiseLossMetric
    })

    ecal_x = X[offset:offset+1]
    ecal_y = y[offset:offset+1]
    ecal_misc = misc[offset:offset+1]
    evaluator = Evaluator(X=ecal_x,y=ecal_y,misc=ecal_misc)
    evaluator.load_model(model)
    evaluator.predict()
    evaluator.cluster(tB, tD)
    return evaluator

# Extract hipo bank information for a specific event
def retreive_hipo_bank_info(hipofile, event_num):
    file = hipopy.hipopy.open(hipofile, mode='r')
    for i,event in enumerate(file):
        if i!=event_num:
            continue
        RECpx = np.array(file.getFloats("REC::Particle","px"))
        RECpy = np.array(file.getFloats("REC::Particle","py"))
        RECpz = np.array(file.getFloats("REC::Particle","pz"))
        RECpid = np.array(file.getInts("REC::Particle","pid"))
        ecal_x = np.array(file.getFloats("REC::Calorimeter","x"))
        ecal_y = np.array(file.getFloats("REC::Calorimeter","y"))
        ecal_sector = np.array(file.getBytes("REC::Calorimeter","sector"))
        ecal_layer  = np.array(file.getBytes("REC::Calorimeter","layer"))
        ecal_pindex = np.array(file.getShorts("REC::Calorimeter","pindex"))
        RECm = np.array([Particle.from_pdgid(p).mass if (p!=0 and p!=45) else 0 for p in RECpid])/1000
        RECall=[ROOT.TLorentzVector(px,py,pz,np.sqrt(px**2+py**2+pz**2+m**2)) for [px,py,pz,m] in zip(RECpx,RECpy,RECpz,RECm)]
        RECgammas=[ROOT.TLorentzVector(px,py,pz,np.sqrt(px**2+py**2+pz**2)) for [px,py,pz,pid] in zip(RECpx,RECpy,RECpz,RECpid) if pid==22]
        RECneutralhadrons=[ROOT.TLorentzVector(px,py,pz,np.sqrt(px**2+py**2+pz**2+m**2)) for [px,py,pz,m,pid] in zip(RECpx,RECpy,RECpz,RECm,RECpid) if (pid==2112)]
        RECchargedhadrons=[ROOT.TLorentzVector(px,py,pz,np.sqrt(px**2+py**2+pz**2+m**2)) for [px,py,pz,m,pid] in zip(RECpx,RECpy,RECpz,RECm,RECpid) if (pid==211 or pid==-211 or pid==321 or pid==-321 or pid==2212)]
        RECe=[ROOT.TLorentzVector(px,py,pz,np.sqrt(px**2+py**2+pz**2+m**2)) for [px,py,pz,m,pid] in zip(RECpx,RECpy,RECpz,RECm,RECpid) if pid==11]
        
        MCpx = np.array(file.getFloats("MC::Lund","px"))
        MCpy = np.array(file.getFloats("MC::Lund","py"))
        MCpz = np.array(file.getFloats("MC::Lund","pz"))
        MCpid = np.array(file.getInts("MC::Lund","pid"))
        MCtype = np.array(file.getBytes("MC::Lund","type"))
        MCparentid = np.array(file.getBytes("MC::Lund","parent"))
        MCfromPi0 = (MCpid[MCparentid-1]==111)
        MCm = np.array([Particle.from_pdgid(p).mass if (p!=0 and p!=91 and p!=92) else 0 for p in MCpid]).astype(float)/1000
        MCgammas=[ROOT.TLorentzVector(px,py,pz,np.sqrt(px**2+py**2+pz**2)) for [px,py,pz,pid,TYPE] in zip(MCpx,MCpy,MCpz,MCpid,MCtype) if pid==22 and TYPE==1]
        
        MCpi0gammas=[ROOT.TLorentzVector(px,py,pz,np.sqrt(px**2+py**2+pz**2)) for [px,py,pz,pid,fromPi0,TYPE] in zip(MCpx,MCpy,MCpz,MCpid,MCfromPi0,MCtype) if (pid==22 and fromPi0==True) and TYPE==1]
        MCneutralhadrons=[ROOT.TLorentzVector(px,py,pz,np.sqrt(px**2+py**2+pz**2+m**2)) for [px,py,pz,m,pid,TYPE] in zip(MCpx,MCpy,MCpz,MCm,MCpid,MCtype) if (pid==2112) and TYPE==1]
        MCchargedhadrons=[ROOT.TLorentzVector(px,py,pz,np.sqrt(px**2+py**2+pz**2+m**2)) for [px,py,pz,m,pid,TYPE] in zip(MCpx,MCpy,MCpz,MCm,MCpid,MCtype) if (pid==211 or pid==-211 or pid==321 or pid==-321 or pid==2212) and TYPE==1]
        MCe=[ROOT.TLorentzVector(px,py,pz,np.sqrt(px**2+py**2+pz**2+m**2)) for [px,py,pz,m,pid,TYPE] in zip(MCpx,MCpy,MCpz,MCm,MCpid,MCtype) if pid==11 and TYPE==1 ]
        break

    bank_info = {
        "RECall": RECall,
        "RECe": RECe,
        "RECgammas": RECgammas,
        "RECneutralhadrons": RECneutralhadrons,
        "RECchargedhadrons": RECchargedhadrons,
        "MCe": MCe,
        "MCgammas": MCgammas,
        "MCpi0gammas": MCpi0gammas,
        "MCneutralhadrons": MCneutralhadrons,
        "MCchargedhadrons": MCchargedhadrons,
        "ecal_pindex": ecal_pindex
    }

    return filter_bank_info(bank_info)

# Filter out TLorentzVector elements with theta < 5 or theta > 35 degrees
def filter_bank_info(bank_info):
    def filter_vectors(vectors):
        return [vec for vec in vectors if 5 <= math.degrees(vec.Theta()) <= 35]

    # Apply the filter to each list of TLorentzVectors in bank_info
    modified_bank_info = {
        "RECall": bank_info["RECall"],
        "RECe": filter_vectors(bank_info["RECe"]),
        "RECgammas": filter_vectors(bank_info["RECgammas"]),
        "RECneutralhadrons": filter_vectors(bank_info["RECneutralhadrons"]),
        "RECchargedhadrons": filter_vectors(bank_info["RECchargedhadrons"]),
        "MCe": filter_vectors(bank_info["MCe"]),
        "MCgammas": filter_vectors(bank_info["MCgammas"]),
        "MCpi0gammas": filter_vectors(bank_info["MCpi0gammas"]),
        "MCneutralhadrons": filter_vectors(bank_info["MCneutralhadrons"]),
        "MCchargedhadrons": filter_vectors(bank_info["MCchargedhadrons"]),
        "ecal_pindex": bank_info["ecal_pindex"]
    }

    return modified_bank_info

def check_particle_counts(bank_info, criteria):
    """
    Check if the counts of particles in bank_info match the given criteria.

    Args:
    - bank_info (dict): Dictionary containing lists of TLorentzVectors for different particle types.
    - criteria (dict): Dictionary with keys as particle types (e.g., "RECe", "RECgammas")
                       and values as [min, max] list specifying the required range of particles.

    Returns:
    - bool: True if all criteria are met, False otherwise.
    """
    for particle_type, (min_count, max_count) in criteria.items():
        if particle_type in bank_info:
            particle_count = len(bank_info[particle_type])
            # Check if the particle count is within the specified min-max range
            if not (min_count <= particle_count <= max_count):
                return False
        else:
            # If the particle_type doesn't exist in bank_info, return False if the min count is > 0
            if min_count > 0:
                return False

    # If no criteria or all criteria are met, return True
    return True


def hunt_event(coatjava_hipofile, objectcondensation_hipofile, criteria, skip=0, version="coatjava"):
    assert(version in ["coatjava","objectcondensation"])
    assert(type(criteria)==dict)

    if version=="coatjava":
        hipofile = coatjava_hipofile
    else:
        hipofile = objectcondensation_hipofile
        
    for event_num in range(1000):
        if event_num<skip:
            continue
        bank_info = retreive_hipo_bank_info(hipofile = hipofile,
                                            event_num = event_num)
        if check_particle_counts(bank_info,criteria):
            return event_num

    print("ERROR: No valid event found that meets the criteria")
    return -1

In [38]:
# Plot the (Theta,Phi) graph
def plot_theta_phi_points(ax,bank_info):
    RECe = bank_info["RECe"]
    RECgammas = bank_info["RECgammas"]
    RECneutralhadrons = bank_info["RECneutralhadrons"]
    RECchargedhadrons = bank_info["RECchargedhadrons"]
    MCe = bank_info["MCe"]
    MCgammas = bank_info["MCgammas"]
    MCpi0gammas = bank_info["MCpi0gammas"]
    MCneutralhadrons = bank_info["MCneutralhadrons"]
    MCchargedhadrons = bank_info["MCchargedhadrons"]

    # Placeholder for each label to ensure the legend entries appear even if no points are plotted
    legend_ms = 10
    ax.plot([-999], [-999], "^", color="darkred", alpha=0.8, label="$e_{MC}$", markeredgecolor="k",markersize=legend_ms)
    ax.plot([-999], [-999], "^", color="darkblue", alpha=0.8, label="$\gamma_{MC}$", markeredgecolor="k",markersize=legend_ms)
    ax.plot([-999], [-999], "^", color="black", alpha=0.8, label="$h^{0}_{MC}$", markeredgecolor="k",markersize=legend_ms)
    ax.plot([-999], [-999], "^", color="darkgreen", alpha=0.8, label="$h^{\pm}_{MC}$", markeredgecolor="k",markersize=legend_ms)
    ax.plot([-999], [-999], "v", color="red", alpha=0.8, label="$e_{REC}$", markeredgecolor="k",markersize=legend_ms)
    ax.plot([-999], [-999], "v", color="cornflowerblue", alpha=0.8, label="$\gamma_{REC}$", markeredgecolor="k",markersize=legend_ms)
    ax.plot([-999], [-999], "v", color="grey", alpha=0.8, label="$h^{0}_{REC}$", markeredgecolor="k",markersize=legend_ms)
    ax.plot([-999], [-999], "v", color="limegreen", alpha=0.8, label="$h^{\pm}_{REC}$", markeredgecolor="k",markersize=legend_ms)


    
    for p in MCe:
        ax.plot([p.Theta()*180/np.pi],[p.Phi()*180/np.pi],"^",markersize=p.P()*4+5,color="darkred",alpha=0.8,markeredgecolor="k")
    for p in MCgammas:
        ax.plot([p.Theta()*180/np.pi],[p.Phi()*180/np.pi],"^",markersize=p.P()*4+5,color="darkblue",alpha=0.8,markeredgecolor="k")
    for p in MCchargedhadrons:
        ax.plot([p.Theta()*180/np.pi],[p.Phi()*180/np.pi],"^",markersize=p.E()*4+5,color="darkgreen",alpha=0.8,markeredgecolor="k")
    for p in MCneutralhadrons:
        ax.plot([p.Theta()*180/np.pi],[p.Phi()*180/np.pi],"^",markersize=p.P()*4+5,color="black",alpha=0.8,markeredgecolor="k")

        
    for p in RECe:
        ax.plot([p.Theta()*180/np.pi],[p.Phi()*180/np.pi],"rv",markersize=p.P()*4+5,alpha=0.8,markeredgecolor="k")        
    for p in RECgammas:
        ax.plot([p.Theta()*180/np.pi],[p.Phi()*180/np.pi],"v",markersize=p.P()*4+5,color="cornflowerblue",alpha=0.8,markeredgecolor="k")
    for p in RECchargedhadrons:
        ax.plot([p.Theta()*180/np.pi],[p.Phi()*180/np.pi],"v",markersize=p.E()*4+5,color="limegreen",alpha=0.8,markeredgecolor="k")
    for p in RECneutralhadrons:
        ax.plot([p.Theta()*180/np.pi],[p.Phi()*180/np.pi],"v",markersize=p.P()*4+5,color="grey",alpha=0.8,markeredgecolor="k")


    
    legend = ax.legend(loc='best',frameon=True, ncol=2,fontsize=8)
    ax.set_xlim(5,35)
    ax.set_ylim(-180,180)
    ax.set_xlabel("Lab Theta [$\\theta$]",fontsize=14)
    ax.set_ylabel("Lab Phi   [$\phi$]",fontsize=14)
    frame  = legend.get_frame()
    frame.set_edgecolor('black')

In [35]:
def event_plot(project_name="", suffix="", model_load_path="", criteria={}, skip=0):
    coatjava_hipofile, objectcondensation_hipofile, objectcondensation_h5file = get_files(project_name=project_name,
                                                                                          suffix=suffix)

    event_num = hunt_event(coatjava_hipofile, objectcondensation_hipofile, criteria, skip)
    print("Event Number -->", event_num, "passes desired criteria...")
    if criteria != {}:
        display(criteria)

    coatjavabank_info = retreive_hipo_bank_info(hipofile=coatjava_hipofile, event_num=event_num)

    objectcondensationbank_info = retreive_hipo_bank_info(hipofile=objectcondensation_hipofile, event_num=event_num)

    fig,axs = plt.subplots(2,3,figsize=(15,10),dpi=150)
    ax0=axs[0][0]
    ax1=axs[0][1]
    ax2=axs[0][2]
    fig.delaxes(axs[1][0])
    ax3=axs[1][1]
    ax4=axs[1][2]
    evaluator = get_evaluator(objectcondensation_h5file, model_load_path, offset=event_num)
    
    ecal_plotter = ModelEcalPlotter(evaluator.get_event_dataframe(event_num), use_clas_calo_scale=True)

    ecal_plotter.plot_mc_peaks(ax0)
    ax0.set_title("Monte Carlo Truth Level",fontsize=18)
    ax0.legend(frameon=True, ncols=2, bbox_to_anchor=(0.5, -0.15), loc='upper center')
    
    plot_theta_phi_points(ax1, coatjavabank_info)
    ax1.set_title("Coatjava",fontsize=15)


    plot_theta_phi_points(ax2, objectcondensationbank_info)
    ax2.set_title("Object Condensation",fontsize=15)

    # Bottom left plot in the original layout
    ecal_plotter.plot_rec_peaks(ax=ax3)
    ax3.legend(frameon=True, ncols=2, bbox_to_anchor=(0.5, -0.15), loc='upper center')

    # Bottom right plot in the original layout
    ecal_plotter.plot_clustered_ecal_peaks(ax=ax4)

    #plt.subplots_adjust(left=0.15, right=0.85, wspace=0.3)  # Adjust the right margin and space between subplots

    #plt.tight_layout()
    plt.show()
    

# User-Defined Section

In [40]:
project_name    = "sadpilot.10.07.2024.12.48"
suffix          = "0000"
model_load_path = "../../projects/sadpilot.10.07.2024.12.48/tensorflow/episode_0001/config_0000/10_08_2024_0000/trained_model.keras"

criteria_0 = {}
criteria_1 = {"RECe": [1, 1], "RECneutralhadrons": [5,10]}


event_plot(project_name    = project_name,
           suffix          = suffix,
           model_load_path = model_load_path,
           criteria        = criteria_1,
           skip            = 3)

compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type = 0 data length = 25839
compression type


KeyboardInterrupt



In [46]:
dst_dir = f'../../projects/{project_name}/dst/'
predict_dir = f'../../projects/{project_name}/predict/'
training_dir = f'../../projects/{project_name}/training/'
file_basename_prefix = "clasdispr.00.e10.600.emn0.75tmn.09.xs68.34nb.dis"
file_basename = f'{file_basename_prefix}.{suffix}'

coatjava_hipofile = f'{dst_dir}/{file_basename}.hipo'
objectcondensation_hipofile = f'{predict_dir}/{file_basename}_ML.hipo'
objectcondensation_h5file = f'{training_dir}/{file_basename}.h5'

reader = ECALDataReader(objectcondensation_hipofile)
for iev,event in enumerate(reader.file):
    if iev!=175:
        continue
    ecal_clusters = reader.get_dict("REC::Calorimeter")
    display(reader.get_dict("REC::Particle"))
    display(ecal_clusters)
    x = ecal_clusters.x
    y = ecal_clusters.y
    z = ecal_clusters.z
    theta  = np.arctan(np.sqrt(x**2+y**2)/z)*180/np.pi
    phi    = np.arctan2(y,x)*180/np.pi
    break

,pid,px,py,pz,vx,vy,vz,vt,charge,beta,chi2pid,status
0,-211,0.058020,0.922481,2.345139,0.471397,-0.047106,0.79363,124.026466,-1,0.998471,-0.470013,-2121
1,22,0.359782,-0.462960,5.818473,0.471397,-0.047106,0.79363,-1.000000,0,1.010144,9999.000000,2040
2,22,0.322858,1.183044,3.764052,0.471397,-0.047106,0.79363,-1.000000,0,0.956078,9999.000000,2020
3,22,-0.075645,-0.425063,0.706685,0.471397,-0.047106,0.79363,-1.000000,0,1.042610,9999.000000,2030
4,2112,-0.136806,-0.223133,0.504176,0.471397,-0.047106,0.79363,-1.000000,0,0.517389,9999.000000,2020


,index,pindex,detector,sector,layer,energy,time,path,chi2,x,y,z,hx,hy,hz,lu,lv,lw,du,dv,dw,m2u,m2v,m2w,m3u,m3v,m3w,status
0,1,0,7,2,4,0.192976,149.612915,760.842834,0.0,58.510162,212.583145,725.651672,58.547882,208.935699,727.896973,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2,0,7,2,7,0.025677,149.783691,779.939209,0.0,61.918564,208.319290,747.381287,59.687256,212.801163,746.563293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,5,1,7,6,1,0.655675,149.028870,757.156250,0.0,47.053772,-59.988346,754.134644,47.053772,-59.988346,754.134644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,6,1,7,6,4,0.171638,150.220154,791.321960,0.0,47.097065,-79.651192,782.075073,47.978542,-70.981888,782.710388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,7,1,7,6,7,0.006752,151.953217,812.430481,0.0,61.690002,-96.964478,792.524597,55.664612,-80.139862,794.235840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
5,0,2,7,2,1,0.501862,149.244125,722.801270,0.0,59.419357,215.955307,688.041626,59.419357,215.955307,688.041626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6,3,3,7,5,1,0.093591,147.355255,729.179810,0.0,-66.134346,-374.319672,623.037109,-66.134346,-374.319672,623.037109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
7,4,3,7,5,7,0.010803,149.529358,787.306213,0.0,-86.788574,-416.888123,656.801086,-78.861435,-409.089905,662.340149,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8,8,4,7,5,1,0.011832,170.234100,716.723999,0.0,-172.136139,-281.572632,636.909424,-172.136139,-281.572632,636.909424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


compression type = 0 data length = 25839


In [47]:
phi

0     74.611272
1     73.446517
2    -51.890005
3    -59.404540
4    -57.534976
5     74.615943
6   -100.019548
7   -101.759975
8   -121.439043
dtype: float64

In [48]:
theta

0    16.901325
1    16.213589
2     5.772821
3     6.747746
4     8.251023
5    18.031812
6    31.387508
7    32.956697
8    27.391400
dtype: float64

In [49]:
evaluator = get_evaluator(h5file=objectcondensation_h5file, model_load_path=model_load_path, tB=0.5, tD=0.5, offset=175)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 113.87it/s]


In [50]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
df = evaluator.dataframe
df = df.sort_values(by='unique_otid')

In [51]:
evaluator.clusters_df

,event,status,sector,layer,energy,time,x,y,z,widthU,widthV,widthW,idU,idV,idW,coordU,coordV,coordW,uid
0,0,0,2,1,0.501862,149.244127,59.419357,215.955307,688.041626,0,0,0,0,0,0,0,0,0,0.0
1,0,0,2,4,0.192976,149.612914,58.510162,212.583145,725.651672,0,0,0,0,0,0,0,0,0,0.0
2,0,0,2,7,0.025677,149.783698,61.918564,208.319290,747.381287,0,0,0,0,0,0,0,0,0,0.0
3,0,1,5,1,0.093591,147.355259,-66.134346,-374.319672,623.037109,0,0,0,0,0,0,0,0,0,1.0
4,0,1,5,7,0.010803,149.529365,-86.788574,-416.888123,656.801086,0,0,0,0,0,0,0,0,0,1.0
5,0,2,6,1,0.655675,149.028864,47.053772,-59.988346,754.134644,0,0,0,0,0,0,0,0,0,2.0
6,0,2,6,4,0.171638,150.220155,47.097065,-79.651192,782.075073,0,0,0,0,0,0,0,0,0,2.0
7,0,2,6,7,0.006752,151.953211,61.690002,-96.964478,792.524597,0,0,0,0,0,0,0,0,0,2.0
8,0,3,5,1,0.011832,170.234106,-172.136139,-281.572632,636.909424,0,0,0,0,0,0,0,0,0,3.0


In [52]:
ecal_clusters.sort_values(by="status")

,index,pindex,detector,sector,layer,energy,time,path,chi2,x,y,z,hx,hy,hz,lu,lv,lw,du,dv,dw,m2u,m2v,m2w,m3u,m3v,m3w,status
0,1,0,7,2,4,0.192976,149.612915,760.842834,0.0,58.510162,212.583145,725.651672,58.547882,208.935699,727.896973,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2,0,7,2,7,0.025677,149.783691,779.939209,0.0,61.918564,208.319290,747.381287,59.687256,212.801163,746.563293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5,0,2,7,2,1,0.501862,149.244125,722.801270,0.0,59.419357,215.955307,688.041626,59.419357,215.955307,688.041626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6,3,3,7,5,1,0.093591,147.355255,729.179810,0.0,-66.134346,-374.319672,623.037109,-66.134346,-374.319672,623.037109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
7,4,3,7,5,7,0.010803,149.529358,787.306213,0.0,-86.788574,-416.888123,656.801086,-78.861435,-409.089905,662.340149,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,5,1,7,6,1,0.655675,149.028870,757.156250,0.0,47.053772,-59.988346,754.134644,47.053772,-59.988346,754.134644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,6,1,7,6,4,0.171638,150.220154,791.321960,0.0,47.097065,-79.651192,782.075073,47.978542,-70.981888,782.710388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,7,1,7,6,7,0.006752,151.953217,812.430481,0.0,61.690002,-96.964478,792.524597,55.664612,-80.139862,794.235840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
8,8,4,7,5,1,0.011832,170.234100,716.723999,0.0,-172.136139,-281.572632,636.909424,-172.136139,-281.572632,636.909424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
